In [14]:
import math

In [15]:
# Helper function to generate a 2D Gaussian kernel
def generate_gaussian_kernel(size, sigma):
    kernel = [[0] * size for _ in range(size)]
    center = size // 2
    total = 0
    for x in range(size):
        for y in range(size):
            diff = ((x - center) ** 2 + (y - center) ** 2) / (2 * sigma * sigma)
            kernel[x][y] = math.exp(-diff) / (2 * math.pi * sigma * sigma)
            total += kernel[x][y]
    
    # Normalize the kernel
    for x in range(size):
        for y in range(size):
            kernel[x][y] /= total
    
    return kernel

In [16]:
# 1. Gaussian Blur
def gaussian_blur(image, sigma):
    size = int(6 * sigma + 1)  # kernel size (3*sigma on both sides)
    if size % 2 == 0:
        size += 1  # make sure size is odd
    kernel = generate_gaussian_kernel(size, sigma)
    height = len(image)
    width = len(image[0])
    
    blurred_image = [[0] * width for _ in range(height)]
    
    # Convolution
    offset = size // 2
    for i in range(offset, height - offset):
        for j in range(offset, width - offset):
            total = 0
            for kx in range(-offset, offset + 1):
                for ky in range(-offset, offset + 1):
                    total += kernel[kx + offset][ky + offset] * image[i + kx][j + ky]
            blurred_image[i][j] = total
    
    return blurred_image

In [17]:

# 2. Bilinear Interpolation for Upsampling
def bilinear_interpolation_upsample(image, scale_factor=2):
    original_height = len(image)
    original_width = len(image[0])
    
    new_height = int(original_height * scale_factor)
    new_width = int(original_width * scale_factor)
    
    upsampled_image = [[0] * new_width for _ in range(new_height)]
    
    for y in range(new_height):
        for x in range(new_width):
            # Map the new pixel to the original pixel coordinates
            orig_y = y / scale_factor
            orig_x = x / scale_factor
            
            # Get the surrounding pixels
            y0 = int(orig_y)
            x0 = int(orig_x)
            y1 = min(y0 + 1, original_height - 1)
            x1 = min(x0 + 1, original_width - 1)
            
            # Calculate interpolation weights
            dy = orig_y - y0
            dx = orig_x - x0
            
            # Perform bilinear interpolation
            upsampled_image[y][x] = (1 - dy) * (1 - dx) * image[y0][x0] + \
                                    dy * (1 - dx) * image[y1][x0] + \
                                    (1 - dy) * dx * image[y0][x1] + \
                                    dy * dx * image[y1][x1]
    
    return upsampled_image

In [18]:
# 3. Difference of Gaussians (DoG)
def difference_of_gaussians(image, sigma1, sigma2):
    blur1 = gaussian_blur(image, sigma1)
    blur2 = gaussian_blur(image, sigma2)
    
    height = len(image)
    width = len(image[0])
    
    dog = [[0] * width for _ in range(height)]
    for i in range(height):
        for j in range(width):
            dog[i][j] = blur2[i][j] - blur1[i][j]
    
    return dog


In [19]:
# 4. Non-Maximum Suppression
def non_max_suppression(dog, threshold):
    height = len(dog)
    width = len(dog[0])
    
    keypoints = []
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            # Check if the point is a local maximum/minimum in a 3x3 neighborhood
            region = [dog[y + dy][x + dx] for dx in range(-1, 2) for dy in range(-1, 2)]
            max_value = max(region)
            min_value = min(region)
            
            if dog[y][x] == max_value and max_value > threshold:
                keypoints.append((x, y))
            elif dog[y][x] == min_value and min_value < -threshold:
                keypoints.append((x, y))
    
    return keypoints

In [20]:
# 5. Gradient Calculation for Descriptors
def compute_gradients(image):
    height = len(image)
    width = len(image[0])
    
    grad_x = [[0] * width for _ in range(height)]
    grad_y = [[0] * width for _ in range(height)]
    
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            grad_x[y][x] = image[y][x + 1] - image[y][x - 1]
            grad_y[y][x] = image[y + 1][x] - image[y - 1][x]
    
    return grad_x, grad_y


In [21]:
# Calculate magnitudes and orientations for each keypoint
def calculate_magnitude_orientation(grad_x, grad_y):
    height = len(grad_x)
    width = len(grad_x[0])
    
    magnitude = [[0] * width for _ in range(height)]
    orientation = [[0] * width for _ in range(height)]
    
    for y in range(height):
        for x in range(width):
            magnitude[y][x] = math.sqrt(grad_x[y][x] ** 2 + grad_y[y][x] ** 2)
            orientation[y][x] = math.degrees(math.atan2(grad_y[y][x], grad_x[y][x]))
    
    return magnitude, orientation



In [22]:
# Descriptor Calculation
def compute_sift_descriptor(image, keypoints, patch_size=16, num_bins=8):
    grad_x, grad_y = compute_gradients(image)
    magnitude, orientation = calculate_magnitude_orientation(grad_x, grad_y)
    
    descriptors = []
    
    for (x, y) in keypoints:
        hist = [0] * num_bins
        for dy in range(-patch_size // 2, patch_size // 2):
            for dx in range(-patch_size // 2, patch_size // 2):
                new_x, new_y = x + dx, y + dy
                if 0 <= new_x < len(image[0]) and 0 <= new_y < len(image):
                    bin_index = int((orientation[new_y][new_x] + 180) / (360 / num_bins)) % num_bins
                    hist[bin_index] += magnitude[new_y][new_x]
        
        # Normalize histogram
        norm = math.sqrt(sum([h ** 2 for h in hist]))
        if norm != 0:
            hist = [h / norm for h in hist]
        
        descriptors.append(hist)
    
    return descriptors



In [23]:
# Main SIFT pipeline from scratch
def sift_pipeline(image, sigma1=1.6, sigma2=3.0, scale_factor=2, threshold=0.03):
    # 1. Gaussian Blurring and DoG
    dog = difference_of_gaussians(image, sigma1, sigma2)
    
    # 2. Upsample the image (optional)
    upsampled_image = bilinear_interpolation_upsample(image, scale_factor)
    
    # 3. Non-Maximum Suppression to get keypoints
    keypoints = non_max_suppression(dog, threshold)
    
    # 4. Compute descriptors
    descriptors = compute_sift_descriptor(upsampled_image, keypoints)
    
    return keypoints, descriptors

# Example usage
image = [[255] * 10 for _ in range(10)]  # Simple 10x10 image
keypoints, descriptors = sift_pipeline(image)
print(f"Detected {len(keypoints)} keypoints.")
print("Descriptors:", descriptors)

Detected 0 keypoints.
Descriptors: []
